In [4]:
import open3d as o3d
import glob
import os
import numpy as np
from utils import showPointCloud,surfaceVertices2WatertightO3dMesh,exportTriMeshObj
import copy

In [2]:
def visualizeObjFile(objFile):
    __mesh = o3d.io.read_triangle_mesh(objFile)
    __mesh.paint_uniform_color(np.array([0.7, 0.7, 0.7]))
    o3d.visualization.draw_geometries([__mesh], window_name=os.path.basename(objFile), width=800, height=600, left=50,
                                      top=50, point_show_normal=True, mesh_show_wireframe=True, mesh_show_back_face=True)
    

In [6]:
# txtFiles = glob.glob(os.path.join(r"F:\Smartee\data\ssa-repaired-txt\33", "3*.txt"))
# txtFiles = glob.glob(os.path.join(r"F:\Smartee\data\repaired-txt\33", "3*.txt"))
# txtFiles = glob.glob(os.path.join(r"F:\Smartee\data\cpdAlignedData\26", "*.txt"))
txtFiles = glob.glob(os.path.join(r"F:\Smartee\dataWithPhoto\format-txt\27", "80U.txt"))
txtFiles = glob.glob(os.path.join(r"F:\Smartee\res", "*1.txt"))
Xs = [np.loadtxt(f) for f in txtFiles]
for x,f in zip(Xs,txtFiles):
    showPointCloud(x, f)

In [4]:
# srcDir = r"F:\Smartee\data-refined\repaired-obj\13"
# objFiles = glob.glob(os.path.join(srcDir,"*.obj"))
# for objFile in objFiles:
#     visualizeObjFile(objFile)

## Visualization of mean upper/lower tooth row of SSM

In [2]:
def loadAlignedPointGroupsWithIndex(loadDir):
    txtFiles = glob.glob(os.path.join(loadDir, "*.txt"))
    txtFileIndices = [os.path.basename(f).split('.')[0] for f in txtFiles]
    alignedPointGroups = [np.loadtxt(f) for f in txtFiles]
    return alignedPointGroups, txtFileIndices

In [3]:
meanToothPGs = []
upperToothIndices = [11,12,13,14,15,16,17,21,22,23,24,25,26,27] #不考虑智齿18,28
lowerToothIndices = [31,32,33,34,35,36,37,41,42,43,44,45,46,47]
toothIndices = upperToothIndices + lowerToothIndices
for toothIndex in toothIndices:
    srcDir = r"./data/cpdGpAlignedData/{}/".format(toothIndex)
    alignedPointGroups, alignedPgIndices = loadAlignedPointGroupsWithIndex(srcDir)
    meanAlignedPG = np.array(alignedPointGroups).mean(axis=0)
    print(toothIndex,meanAlignedPG.shape)
    meanToothPGs.append(meanAlignedPG)
    

11 (1500, 3)
12 (1500, 3)
13 (1500, 3)
14 (1500, 3)
15 (1500, 3)
16 (1500, 3)
17 (1500, 3)
21 (1500, 3)
22 (1500, 3)
23 (1500, 3)
24 (1500, 3)
25 (1500, 3)
26 (1500, 3)
27 (1500, 3)
31 (1500, 3)
32 (1500, 3)
33 (1500, 3)
34 (1500, 3)
35 (1500, 3)
36 (1500, 3)
37 (1500, 3)
41 (1500, 3)
42 (1500, 3)
43 (1500, 3)
44 (1500, 3)
45 (1500, 3)
46 (1500, 3)
47 (1500, 3)


In [7]:
meanUpperToothRowPoints = np.vstack(meanToothPGs[:14])
showPointCloud(meanUpperToothRowPoints, "mean upper tooth row")

In [8]:
meanLowerToothRowPoints = np.vstack(meanToothPGs[14:])
showPointCloud(meanLowerToothRowPoints, "mean lower tooth row")

In [9]:
meanToothMeshes = [surfaceVertices2WatertightO3dMesh(pg) for pg in meanToothPGs]

In [10]:
def mergeO3dTriangleMeshes(o3dMeshes):
    """合并多个Triangle Meshes"""
#     newVStartIndex = 0
#     aggVertices = []
#     aggTriangles = []
#     for _mesh in o3dMeshes:
#         vNum = len(_mesh.vertices)
#         tempVertices = np.asarray(_mesh.vertices)
#         tempTriangles = np.asarray(_mesh.triangles) + newVStartIndex
#         aggVertices.append(tempVertices)
#         aggTriangles.append(tempTriangles)
#         newVStartIndex += vNum
#     aggMesh = o3d.geometry.TriangleMesh()
#     aggMesh.vertices = o3d.utility.Vector3dVector(np.vstack(aggVertices))
#     aggMesh.triangles = o3d.utility.Vector3iVector(np.vstack(aggTriangles))
#     return aggMesh
    _msh = copy.deepcopy(o3dMeshes[0])
    for _tmp_msh in o3dMeshes[1:]:
        _msh += _tmp_msh
    return _msh

In [11]:
meanUpperToothRowMesh = mergeO3dTriangleMeshes(meanToothMeshes[:14])
o3d.visualization.draw_geometries([meanUpperToothRowMesh], window_name="meanUpperToothRowMesh", width=800, height=600, left=50,top=50, point_show_normal=False)
exportTriMeshObj(np.asarray(meanUpperToothRowMesh.vertices), np.asarray(meanUpperToothRowMesh.triangles), "./meanUpperToothRowMesh.obj")

In [12]:
meanLowerToothRowMesh = mergeO3dTriangleMeshes(meanToothMeshes[14:])
o3d.visualization.draw_geometries([meanLowerToothRowMesh], window_name="meanLowerToothRowMesh", width=800, height=600, left=50,top=50, point_show_normal=False)
exportTriMeshObj(np.asarray(meanLowerToothRowMesh.vertices), np.asarray(meanLowerToothRowMesh.triangles), "./meanLowerToothRowMesh.obj")

In [ ]:
# visualization.py

import glob
import os
import numpy as np
import open3d as o3d

def showPredPointClouds(y,py):
    pcdY = o3d.geometry.PointCloud()
    pcdY.points = o3d.utility.Vector3dVector(y)
    pcdY.paint_uniform_color(np.array([1.,0.,0.]))
    pcdPY = o3d.geometry.PointCloud()
    pcdPY.points = o3d.utility.Vector3dVector(py)
    pcdPY.paint_uniform_color(np.array([0.,0.,1.]))
    o3d.visualization.draw_geometries([pcdY,pcdPY], window_name="Ground Truth PCL(red) and Prediction PCL(blue)", width=800, height=600, left=50,top=50, point_show_normal=False)

def __tag(f):
    return int(os.path.basename(f).split(".")[0])
    
if __name__ == "__main__":
    saveDir = r"./res/"
    # saveDir = r"./res-2D-3D/"
    groundTruthDir = os.path.join(saveDir, "y_true")
    predictionDir = os.path.join(saveDir, "y_pred")
    yTxtFiles = glob.glob(os.path.join(groundTruthDir, "*.txt"))
    yTxtFiles = sorted(yTxtFiles, key=lambda f:__tag(f))
    pyTxtFiles = glob.glob(os.path.join(predictionDir, "*.txt"))
    pyTxtFiles = sorted(pyTxtFiles, key=lambda f:__tag(f))
    # print(yTxtFiles)
    # print(pyTxtFiles)
    ys = [np.loadtxt(f) for f in yTxtFiles]
    pys = [np.loadtxt(f) for f in pyTxtFiles]
    for y,py in zip(ys, pys):
        showPredPointClouds(y, py)